In [1]:
from pathlib import Path
from dataclasses import dataclass
import numpy as np

In [2]:
@dataclass
class Robot:
    x: int
    y: int
    v_x: int
    v_y: int


def parse(pos, vel):
    x, y = [int(x) for x in pos[2:].split(",")]
    v_x, v_y = [int(x) for x in vel[2:].split(",")]
    return Robot(y, x, v_y, v_x) # note the swap due to np convention


path = Path("data/14.txt")
robots = [parse(*l.split()) for l in path.read_text().strip().split("\n")]

In [3]:
grid = np.full((7, 11) if "sample" in path.name else (103, 101), 0)
w, h = grid.shape
steps = 100

for r in robots:
    x, y = r.x, r.y
    x += steps * r.v_x
    y += steps * r.v_y
    x %= w
    y %= h
    grid[x, y] += 1

q1 = grid[: w // 2, : h // 2]
q2 = grid[: w // 2, h // 2 + 1 :]
q3 = grid[w // 2 + 1 :, : h // 2]
q4 = grid[w // 2 + 1 :, h // 2 + 1 :]
q1.sum() * q2.sum() * q3.sum() * q4.sum()

230686500

In [4]:
import math

def count_within_distance(positions, w, h, dist=2):
    """
    Given a list of (x, y) positions, count the number of robots within 'dist' radius for each robot.
    Return some metric, e.g., the sum or average of neighbors within that distance.
    """
    pos_set = set(positions)
    total_neighbors = 0
    for (x, y) in positions:
        # Count how many robots are within a radius of dist
        # We'll use Euclidean distance here. Check the neighborhood from (x-dist) to (x+dist) and (y-dist) to (y+dist).
        neighbor_count = 0
        for nx in range(x - dist, x + dist + 1):
            for ny in range(y - dist, y + dist + 1):
                # Wrap around if necessary (if you consider wrap-around distance)
                # But since positions are already modulo w,h, no need to re-wrap here
                # Just ensure coordinates are valid if needed
                if (nx, ny) in pos_set:
                    # Compute Euclidean distance
                    dx = nx - x
                    dy = ny - y
                    d = math.sqrt(dx*dx + dy*dy)
                    if d <= dist and not (dx == 0 and dy == 0):
                        neighbor_count += 1
        total_neighbors += neighbor_count
    # For example, return the average number of neighbors
    avg_neighbors = total_neighbors / len(positions)
    return avg_neighbors


def print_grid_at_time(robots, w, h, t):
    # Compute the positions of all robots at time t
    positions = []
    for r in robots:
        cur_x = (r.x + t * r.v_x) % w
        cur_y = (r.y + t * r.v_y) % h
        positions.append((cur_x, cur_y))
    
    min_x = min(pos[0] for pos in positions)
    max_x = max(pos[0] for pos in positions)
    min_y = min(pos[1] for pos in positions)
    max_y = max(pos[1] for pos in positions)
    
    # Create a set for quick lookup
    pos_set = set(positions)
    
    # Print the bounding box region
    for x in range(min_x, max_x + 1):
        line = []
        for y in range(min_y, max_y + 1):
            if (x, y) in pos_set:
                line.append('#')
            else:
                line.append('.')
        print("".join(line))


def track_neighbors_within_2_distance_over_time(robots, w, h, max_time=20000):
    """
    For each time step up to max_time, compute the average number of neighbors within distance 2.
    Track when it's maximal or minimal.
    """
    best_t = None
    best_avg = None
    
    for t in range(max_time):
        positions = []
        for r in robots:
            cur_x = (r.x + t * r.v_x) % w
            cur_y = (r.y + t * r.v_y) % h
            positions.append((cur_x, cur_y))
        
        avg_neighbors = count_within_distance(positions, w, h, dist=2)
        
        # Track when avg_neighbors is smallest or largest depending on what you're looking for
        # If you're looking for times of greatest clustering, maybe you want the largest avg_neighbors.
        
        if best_avg is None or avg_neighbors > best_avg:
            best_avg = avg_neighbors
            best_t = t
            # print(f"Time {t}: avg neighbors within distance 2 = {avg_neighbors}")
    
    print("Maximum average neighbors within distance 2 found at time:", best_t, "avg:", best_avg)
    return best_t, best_avg

In [5]:
best_t, best_avg = track_neighbors_within_2_distance_over_time(robots, w, h, max_time=50000)
print_grid_at_time(robots, w, h, best_t)

Maximum average neighbors within distance 2 found at time: 7672 avg: 5.664
.............................#........................................#.........................#....
#............#...................................#.............#.....................#...............
...................#...............#.................................................................
.......................................................................................#.............
........................................................................#............................
.....#.................................................................................#.............
.....................................................................................................
............................#...................#....................................................
.....................................................................................#...............
.......